This notebook presents the appplication of Bayesian Optimisation to the problem of optimally controlling the 
stretcher parameters of the L1 pump Laser. In particular, in this notebook, considering the results observed in `loss_function.ipynb` a phase of hyperparameter tuning is carried out using the best loss function found.

Author: Francesco Capuano, 2022 S17 summer intern @ ELI-beamlines, Prague

# Setting
The goal of this project is to maximise second-harmonic efficiency. 

However, since this metric is also very much related to the shortest possible pulse shape, we developed a strategy to optimise a predefinite set of control parameters so as to minimise the difference between the obtained pulse shape (in the temporal domain) and a target one (which is, by default, the shortest one).

This process is very much intensive for what concerns the number of function evaluation we needed to use and, because of that, we resorted to use a custom-built fast-running forward model used to approximate/predict the temporal  profile of the pulse given a set of parameters. 

In particular, if one indicates with $\psi = \big( d_1, d_2, d_3 \big)$ the control configuration of the stretcher and with $E(\nu)$ the electric field in the frequency domain of the signal considered, then the fast running model we developed approximates $ \tilde{\tau}(\psi; E(\nu)) $, hopefully similar enough to the actual ${\tau}(\psi; E(\nu))$ that can be collected only through practical observation on the real hardware. 

If one has a target temporal profile $\tau^*$ of the pulse then it the problem can be framed as: 

$$\min_{\psi} L\big[ \tilde{\tau}(\psi; E(\nu)), \tau^*\big]$$

With $L$ being some sort of loss function. 

Our experiments have shown that default Bayesian Optimization performs best if using as loss function the L1-Manhattan norm, defined in this case as: 

$$
L\big[ \tilde{\tau}(\psi; E(\nu)), \tau^*\big] = \sum_{i}  \big\vert \tilde{\tau}(\psi; E(\nu))_i - \tau^*_i \big\vert
$$

Considering the results obtained, we decide to analyse and experiment with this objective function.

In [1]:
# these imports are necessary to import modules from directories one level back in the folder structure
import sys
sys.path.append("../..")

from algorithms.L1_BayesianOptimisation import *
from utils import physics

frequency, intensity = extract_data() # extracting the desired information

# compressor parameters - obtained as minus the stretcher one - TO BE FIXED WITH REAL ONES ONCE THEY ARE AVAILABLE
COMPRESSOR = -1 * np.array((267.422 * 1e-24, -2.384 * 1e-36, 9.54893 * 1e-50)) # in s^2, s^3 and s^4 (SI units)
# non linearity parameter
B = 2
# cutoff frequencies, in THz, used to remove noise - derived from visual inspection
CUTOFF = (289.95, 291.91)
# model instantiation
l1_pump = model(frequency, intensity, COMPRESSOR, B, CUTOFF, num_points=int(5e3))
# pre-processed version of frequency and intensity
frequency_clean, intensity_clean = l1_pump.spit_center()

# target temporal profile shape
_, target_profile = physics.temporal_profile(frequency_clean, np.sqrt(intensity_clean),
                                             phase = np.zeros_like(frequency_clean),
                                             npoints_pad = l1_pump.pad_points)

temporal_profile = lambda d2, d3, d4: l1_pump.forward_pass(np.array((d2, d3, d4)))[1]

GDDperc, TODperc, FODperc = 0.1, 0.2, 0.3
# stretcher control bounds are centered in the compressor bounds and have a width related to a given percentage
# (which can be though of as an hyperparameter as long as it is in the tunable interval)

low_stretcher, high_stretcher = (-1 * COMPRESSOR * np.array((1 - GDDperc, 1 - TODperc, 1 - FODperc)), 
                                 -1 * COMPRESSOR * np.array((1 + GDDperc, 1 + TODperc, 1 + FODperc))
                                )

# stretcher control must be given in terms of dispersion coefficients so they must be translated into d2, d3 and d4. 
low_stretcher, high_stretcher = (l1_pump.translate_control(low_stretcher, verse = "to_disp"),
                                 l1_pump.translate_control(high_stretcher, verse = "to_disp")
                                )

# these are the bounds for the parameter currently optimized - sign can change so sorting is used
pbounds = {
    "d2": np.sort((low_stretcher[0], high_stretcher[0])), 
    "d3": np.sort((low_stretcher[1], high_stretcher[1])), 
    "d4": np.sort((low_stretcher[2], high_stretcher[2]))
}

In [2]:
#loss-6
def objective_function(d2, d3, d4): 
    tol = 1e-6 # zero tolerance
    
    controlled_profile = temporal_profile(d2, d3, d4)
    mask = (controlled_profile != target_profile) & (controlled_profile > tol)
    
    controlled_profile = controlled_profile[mask]
    target = target_profile[mask]
    
    return -1 * (np.abs(controlled_profile - target)).sum()